In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_8bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "../model/Qwen2.5-7B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = False,
    load_in_8bit = load_in_8bit,
    gpu_memory_utilization=1,
    device_map='auto'
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


[4pdvGPU Msg(71351:139774776604480:libvgpu.c:869)]: Initializing.....
[4pdvGPU Warn(71351:139774776604480:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(71351:139774776604480:hook.c:475)]: remap handles for device 1
[4pdvGPU Warn(71351:139774776604480:utils.c:228)]: get default cuda 2 from (null)
[4pdvGPU Msg(71351:139774776604480:libvgpu.c:902)]: Initialized


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/root/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


[4pdvGPU Msg(72908:140316252411712:libvgpu.c:869)]: Initializing.....
[4pdvGPU Warn(72908:140316252411712:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(72908:140316252411712:hook.c:475)]: remap handles for device 1
[4pdvGPU Warn(72908:140316252411712:utils.c:228)]: get default cuda 2 from (null)
[4pdvGPU Msg(72908:140316252411712:libvgpu.c:902)]: Initialized


==((====))==  Unsloth 2025.5.9: Fast Qwen2 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A30. Num GPUs = 2. Max memory: 24.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


[4pdvGPU Msg(71351:139774776604480:memory.c:566)]: orig free=24866193408 total=25217466368 limit=25769803776 usage=234881024
[4pdvGPU Msg(71351:139774776604480:memory.c:566)]: orig free=24950079488 total=25217466368 limit=25769803776 usage=234881024
Loading checkpoint shards:   0%|                                        | 0/4 [00:00<?, ?it/s][4pdvGPU Msg(71351:139774776604480:memory.c:566)]: orig free=24866193408 total=25217466368 limit=25769803776 usage=234881024
[4pdvGPU Msg(71351:139774776604480:memory.c:566)]: orig free=24950079488 total=25217466368 limit=25769803776 usage=234881024
Loading checkpoint shards: 100%|████████████████████████████████| 4/4 [00:07<00:00,  1.93s/it]
[4pdvGPU Warn(73112:140436190304064:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(73112:140436190304064:hook.c:475)]: remap handles for device 1


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [3]:
alpaca_prompt = """

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts}
pass

from datasets import load_dataset
dataset = load_dataset("/mnt/proj/data/DISC-Law-SFT/normal/",split="train")
dataset = dataset.shuffle(seed=46)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████████████████████████████| 246450/246450 [00:09<00:00, 24999.89 examples/s]


In [7]:
len(dataset)

214408

In [4]:
dataset[190000]

{'id': 'leg_case_cls-25',
 'input': '以下文本反映了什么类型的风险\n江苏宿迁的盛先生一家,最近很苦恼.为啥?家里40万的卖房款,被两个孩子花得一分不剩,这些钱都让熊孩子给各平台主播打赏了!你说气人不?原来盛先生陪父母到银行办业务,发现银行卡里的40万不翼而飞!这可是元旦前家里卖房子的钱啊.一查银行流水显示,从1月4日到2月16日之间,每天都有三四十笔支付宝或QQ支付转账,每笔资金从几块钱到一千多块钱不等,最多的一天有四五万块钱.40万不是一个小数字,孩子花这么多钱,大人怎么会不知道呢?据盛先生讲,他和弟弟常年在外打工,侄子小北跟着奶奶生活.因为老年机无法下载习题作业,于是给小北买了一部智能手机.本想监督孩子写作业,没想到孩子竟迷上了手游,还用奶奶的手机号注册,并且绑定了银行卡.孩子是如何得知密码的呢?原来奶奶年纪大了,害怕记不住密码,就写在了一张纸上,没想到被孩子发现了.需要输入验证码的时候,拿起老人手机一看便知,而这一切,孩子奶奶毫不知情!于是小北和另外一个13岁的孩子,两人开起狂欢模式,疯狂给主播们打赏.孩子在家没事可干,加上家长疏于监督,玩游戏会导致行为上瘾.经常玩游戏的人都知道,老虎机输的时候沉默无声,赢的时候会有红灯闪烁、好像你赢得了全世界一样.所以才会有那么多的人迷恋游戏,它给人的感官刺激太大了.40万,对于一个农村家庭来说不是小数,真不知道父母要辛苦攒多少年血汗钱,才能凑齐这40万.令人高兴的是,已经有一部分平台答应给小北一家退钱了.希望孩子和家长都能记住这个教训.#两熊孩子玩游戏打赏主播花光40万#',
 'output': '盗用风险',
 'text': '\n\n### Input:\n以下文本反映了什么类型的风险\n江苏宿迁的盛先生一家,最近很苦恼.为啥?家里40万的卖房款,被两个孩子花得一分不剩,这些钱都让熊孩子给各平台主播打赏了!你说气人不?原来盛先生陪父母到银行办业务,发现银行卡里的40万不翼而飞!这可是元旦前家里卖房子的钱啊.一查银行流水显示,从1月4日到2月16日之间,每天都有三四十笔支付宝或QQ支付转账,每笔资金从几块钱到一千多块钱不等,最多的一天有四五万块钱.40万不是一个小数字,孩子花这么多钱,大人怎么会不知道呢?据盛先生讲,他和弟弟常年在外打工,侄子小北跟着奶奶生活.因为老年机无法下载习

In [4]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3409,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

[4pdvGPU Warn(73164:139667537925952:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(73164:139667537925952:hook.c:475)]: remap handles for device 1
[4pdvGPU Warn(73165:140668000573248:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(73165:140668000573248:hook.c:475)]: remap handles for device 1
Unsloth: Tokenizing ["text"]: 100%|██████████| 246450/246450 [01:03<00:00, 3901.88 examples/s]
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [5]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 2
   \\   /|    Num examples = 246,450 | Num Epochs = 1 | Total steps = 3,851
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 8 x 1) = 64
 "-____-"     Trainable parameters = 80,740,352/7,696,356,864 (1.05% trained)
[4pdvGPU Warn(71351:139774776604480:event.c:20)]: cuModuleLoadData module=0x7ffd07f3e698
[4pdvGPU Warn(71351:139774776604480:event.c:20)]: cuModuleLoadData module=0x7ffd07f3d938
[4pdvGPU Warn(71351:139774776604480:event.c:20)]: cuModuleLoadData module=0x7ffd07f3d938
[4pdvGPU Warn(71351:139774776604480:event.c:20)]: cuModuleLoadData module=0x7ffd07f3df78
[4pdvGPU Warn(71351:139774776604480:event.c:20)]: cuModuleLoadData module=0x7ffd07f3e698
[4pdvGPU Warn(71351:139774776604480:event.c:20)]: cuModuleLoadData module=0x7ffd07f3d938
[4pdvGPU Warn(71351:139774776604480:event.c:20)]: cuModuleLoadData module=0x7ffd07f3d938
[4pdvG

Unsloth: Will smartly offload gradients to save VRAM!


[4pdvGPU Warn(71351:139766112708352:event.c:20)]: cuModuleLoadData module=0x7f1dd57f8798
[4pdvGPU Warn(71351:139766112708352:event.c:20)]: cuModuleLoadData module=0x7f1dd57f88a8
[4pdvGPU Warn(71351:139766112708352:event.c:20)]: cuModuleLoadData module=0x7f1dd57f88a8
[4pdvGPU Warn(71351:139766121101056:event.c:20)]: cuModuleLoadData module=0x7f1dd5ff9798
[4pdvGPU Warn(71351:139766121101056:event.c:20)]: cuModuleLoadData module=0x7f1dd5ff98a8
[4pdvGPU Warn(71351:139766121101056:event.c:20)]: cuModuleLoadData module=0x7f1dd5ff98a8
[4pdvGPU Warn(71351:139766121101056:event.c:20)]: cuModuleLoadData module=0x7f1dd5ff98a8
[4pdvGPU Warn(71351:139774776604480:event.c:20)]: cuModuleLoadData module=0x7ffd07f3d938
[4pdvGPU Warn(71351:139774776604480:event.c:20)]: cuModuleLoadData module=0x7ffd07f3d938
[4pdvGPU Warn(71351:139774776604480:event.c:20)]: cuModuleLoadData module=0x7ffd07f3d938
[4pdvGPU Warn(71351:139774776604480:event.c:20)]: cuModuleLoadData module=0x7ffd07f3d938
[4pdvGPU Warn(71351:1

Step,Training Loss
1,1.562300
2,1.697400
3,1.690700
4,1.570900
5,1.695100
6,1.648100
7,1.807700
8,1.815500
9,1.760200
10,1.856100


[4pdvGPU Warn(71351:139766112708352:event.c:20)]: cuModuleLoadData module=0x7f1dd57fbc98
[4pdvGPU Warn(71351:139774776604480:event.c:20)]: cuModuleLoadData module=0x7ffd07f410e8
[4pdvGPU Warn(71351:139774776604480:event.c:20)]: cuModuleLoadData module=0x7ffd07f410e8
[4pdvGPU Warn(71351:139766112708352:event.c:20)]: cuModuleLoadData module=0x7f1dd57fbc98
[4pdvGPU Warn(71351:139766112708352:event.c:20)]: cuModuleLoadData module=0x7f1dd57fbc98
[4pdvGPU Warn(71351:139774776604480:event.c:20)]: cuModuleLoadData module=0x7ffd07f410e8


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model).to("cuda:1") # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "请归纳这篇文书的大致要点 莫振红与齐爱青、龙书贵借款合同纠纷一审民事判决书 河南省滑县人民法院 民 事 判 决 书 （2018）豫0526民初805号原告莫振红，女，1973年3月3日生，汉族。 委托代理人孟帅、李亚廷（实习），河南卓誉律师事务所律师。 被告齐爱青，女，1968年2月28日生，汉族。 被告龙书贵，男，1964年2月3日生，汉族。 原告莫振红诉被告齐爱青、龙书贵借款合同纠纷一案，本院受理后，依法适用简易程序，公开开庭进行了审理。 原告莫振红的委托代理人孟帅、被告齐爱青到庭参加诉讼，被告龙书贵经传票传唤未到庭参加诉讼。 本案现已审理终结。 原告莫振红诉称，2014年11月11日，二被告向原告借款2万元，并向原告出具了借条一份。 后经多次催要，被告均以各种理由推拖没有偿还原告。 起诉请求依法判令被告偿还原告借款2万元及利息； 诉讼费用由被告承担。 被告齐爱青辩称，借款属实，但中间原告拉走3车砖折抵借款0.8万元，尚欠1.2万元，故被告现在不欠原告2万元钱。 被告龙书贵缺席未答辩。 经审理查明：2014年11月11日，被告齐爱青、龙书贵共同向原告借款2万元，并出具借条一份，内容为：“今借到现金贰万元正￥20000元2014年11月11日齐爱青龙书贵。” 上述借款至今未予偿还。 以上事实，有原告提交的借条及原、被告部分庭审陈述可以证实，上述证据经庭审质证，可以作为认定本案事实的依据。 本院认为，被告齐爱青、龙书贵共同向原告借款2万元，有原告提供的被告齐爱青、龙书贵签名确认的借条予以证实，故被告齐爱青、龙书贵负有共同偿还的责任。 关于利息，因未明确约定，应自原告主张权利时即起诉之日起，按年利率6%计算。 被告齐爱青主张原告拉砖三车即已经折抵借款0.8万元及其他辩解意见，均证据不足，依法不予采信。 根据《中华人民共和国民法通则》第八十四条、第一百零八条，《中华人民共和国民事诉讼法》第六十四条、第一百四十四条的规定，判决如下：被告齐爱青、龙书贵于本判决生效后五日内共同偿还原告莫振红借款人民币2万元及利息（自2018年1月22日起，按照年利率6%计算至本判决限定的履行期限届满之日止）。 如果未按本判决指定的期间履行给付金钱义务，应当依照《中华人民共和国民事诉讼法》第二百五十三条之规定，加倍支付迟延履行期间的债务利息。 案件受理费300元，减半收取150元，被告齐爱青、龙书贵共同负担。 如不服本判决，可在判决送达之日起十五日内，向本院递交上诉状，并按当事人的人数提出副本，上诉于河南省安阳市中级人民法院。 审判员　　张兴政 二〇一八年二月二十二日 书记员　　王舒舒", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda:1")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1280)

In [6]:
model.save_pretrained("../model/lora_7b_full_data")  # Local saving
tokenizer.save_pretrained("../model/lora_7b_full_data")

('../model/lora_7b_full_data/tokenizer_config.json',
 '../model/lora_7b_full_data/special_tokens_map.json',
 '../model/lora_7b_full_data/chat_template.jinja',
 '../model/lora_7b_full_data/vocab.json',
 '../model/lora_7b_full_data/merges.txt',
 '../model/lora_7b_full_data/added_tokens.json',
 '../model/lora_7b_full_data/tokenizer.json')

In [7]:
if True: model.save_pretrained_merged("../model/law_7b_full_data_f16", tokenizer, save_method = "merged_16bit")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 320.8 out of 503.19 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|█████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 28.54it/s]


Unsloth: Saving tokenizer... Done.
Done.


In [1]:
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name = "../model/law_3b_full_data_f16"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)



[4pdvGPU Msg(69350:140645045749568:libvgpu.c:869)]: Initializing.....
[4pdvGPU Warn(69350:140645045749568:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(69350:140645045749568:hook.c:475)]: remap handles for device 1
/root/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[4pdvGPU Warn(69350:140645045749568:utils.c:228)]: get default cuda 2 from (null)
[4pdvGPU Msg(69350:140645045749568:libvgpu.c:902)]: Initialized
[4pdvGPU Msg(69350:140645045749568:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
[4pdvGPU Msg(69350:140645045749568:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
Loading checkpoint shards:   0%|                                        | 0/2 [00:00<?, ?it/s][4pdvGPU Msg(69350:1406450

content: 这是一起借款合同纠纷案件，涉及原告莫振红和被告齐爱青、龙书贵。根据案件描述：

原告莫振红诉称被告齐爱青、龙书贵在2014年11月11日向她借款2万元，并出具了一份借条。

被告齐爱青辩称，借款属实，但是原告拉走了三车砖，折抵了借款中的0.8万元，所以目前只欠原告1.2万元。

法院判决如下：
1. 被告齐爱青、龙书贵共同偿还借款2万元；
2. 被告齐爱青、龙书贵共同支付利息（自原告起诉之日起，按照年利率6%计算至判决限定的履行期限届满之日止）。

需要注意的是，被告齐爱青的辩解意见并未被采纳。


In [5]:
# prepare the model input
# prompt = """
# 请归纳这篇文书的大致要点 莫振红与齐爱青、龙书贵借款合同纠纷一审民事判决书 河南省滑县人民法院 民 事 判 决 书 （2018）豫0526民初805号原告莫振红，女，1973年3月3日生，汉族。 委托代
# 理人孟帅、李亚廷（实习），河南卓誉律师事务所律师。 被告齐爱青，女，1968年2月28日生，汉族。 被告龙书贵，男，1964年2月3日生，汉族。 原告莫振红诉被告齐爱青、龙书贵借款合同纠纷一案，本院受理后，
# 依法适用简易程序，公开开庭进行了审理。 原告莫振红的委托代理人孟帅、被告齐爱青到庭参加诉讼，被告龙书贵经传票传唤未到庭参加诉讼。 本案现已审理终结。 原告莫振红诉称，2014年11月11日，二被告向原告
# 借款2万元，并向原告出具了借条一份。 后经多次催要，被告均以各种理由推拖没有偿还原告。 起诉请求依法判令被告偿还原告借款2万元及利息； 诉讼费用由被告承担。 被告齐爱青辩称，借款属实，但中间原告拉走3
# 车砖折抵借款0.8万元，尚欠1.2万元，故被告现在不欠原告2万元钱。 被告龙书贵缺席未答辩。 经审理查明：2014年11月11日，被告齐爱青、龙书贵共同向原告借款2万元，并出具借条一份，内容为：“今借到现金贰万元
# 正￥20000元2014年11月11日齐爱青龙书贵。” 上述借款至今未予偿还。 以上事实，有原告提交的借条及原、被告部分庭审陈述可以证实，上述证据经庭审质证，可以作为认定本案事实的依据。 本院认为，被告齐爱青、
# 龙书贵共同向原告借款2万元，有原告提供的被告齐爱青、龙书贵签名确认的借条予以证实，故被告齐爱青、龙书贵负有共同偿还的责任。 关于利息，因未明确约定，应自原告主张权利时即起诉之日起，按年利率6%计算。 
# 被告齐爱青主张原告拉砖三车即已经折抵借款0.8万元及其他辩解意见，均证据不足，依法不予采信。 根据《中华人民共和国民法通则》第八十四条、第一百零八条，《中华人民共和国民事诉讼法》第六十四条、第一百四十四条的
# 规定，判决如下：被告齐爱青、龙书贵于本判决生效后五日内共同偿还原告莫振红借款人民币2万元及利息（自2018年1月22日起，按照年利率6%计算至本判决限定的履行期限届满之日止）。 如果未按本判决指定的期间履行给付
# 金钱义务，应当依照《中华人民共和国民事诉讼法》第二百五十三条之规定，加倍支付迟延履行期间的债务利息。 案件受理费300元，减半收取150元，被告齐爱青、龙书贵共同负担。 如不服本判决，可在判决送达之日起十五
# 日内，向本院递交上诉状，并按当事人的人数提出副本，上诉于河南省安阳市中级人民法院。 审判员　　张兴政 二〇一八年二月二十二日 书记员　　王舒舒"""
prompt = """
这是一个关于法律问题的单项选择题,请根据题目选择正确的选项(只需要给出正确选项即可)：\n
问题：马锡五在审理“抢亲案"时发现，女方与男方两情相悦，定有婚约，女方父亲为了更多的彩礼将女儿许配另一人，男方父亲带领婚礼抢亲，马锡五判婚姻有效，男方父亲判徒刑半年，女方父亲判劳役半年，
判决一出，群众无不交口称赞。关于马锡五审判，以下哪一说法是错误的？\n
A：群众不能理解的判决，很难具有公信力\n
B:司法裁判要坚持走群众路线\n
C:马锡五知民情知民意\n
D:只要群众满意，不必恪守法律\n
"""
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=50
)

content = tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True).strip("\n")

print("content:", content)

content: D


In [6]:
model_name = "../model/law_3b_f16"

# load the tokenizer and the model
tokenizer_cot = AutoTokenizer.from_pretrained(model_name)
model_cot = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

[4pdvGPU Msg(69350:140645045749568:memory.c:566)]: orig free=21137457152 total=25217466368 limit=25769803776 usage=4018275328
[4pdvGPU Msg(69350:140645045749568:memory.c:566)]: orig free=21670133760 total=25217466368 limit=25769803776 usage=3485598720
Loading checkpoint shards:   0%|                                        | 0/2 [00:00<?, ?it/s][4pdvGPU Msg(69350:140645045749568:memory.c:566)]: orig free=21137457152 total=25217466368 limit=25769803776 usage=4018275328
[4pdvGPU Msg(69350:140645045749568:memory.c:566)]: orig free=21670133760 total=25217466368 limit=25769803776 usage=3485598720
Loading checkpoint shards: 100%|████████████████████████████████| 2/2 [00:16<00:00,  8.23s/it]
[4pdvGPU Warn(69916:140369235285824:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(69916:140369235285824:hook.c:475)]: remap handles for device 1


In [ ]:
import pandas as pd

data = "./mcq_sing_nje.csv"
df = pd.read_csv(data)
answer = []
acc = 0
for i in range(len(df)):
    prompt = f"""
    这是一个关于法律问题的单项选择题,请根据题目选择正确的选项(只需要回答正确选项即可，不需要进行分析)：\n
    问题：{df["input"][i]}\n
    A：{df["A"][i]}\n
    B:{df["B"][i]}\n
    C:{df["C"][i]}\n
    D:{df["D"][i]}\n
    """
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=50
    )
    
    content = tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True).strip("\n")
    print("content:", content)
    if content[0] ==  df["output"][i]:
        acc += 1    
    answer.append(content[0])
print(acc)

In [22]:
df["output"][:10]

0    D
1    D
2    C
3    D
4    B
5    A
6    D
7    C
8    A
9    A
Name: output, dtype: object

In [7]:

prompt = """
这是一个关于法律问题的单项选择题,请根据题目选择正确的选项(只需要给出正确选项即可)：\n
问题：马锡五在审理“抢亲案"时发现，女方与男方两情相悦，定有婚约，女方父亲为了更多的彩礼将女儿许配另一人，男方父亲带领婚礼抢亲，马锡五判婚姻有效，男方父亲判徒刑半年，女方父亲判劳役半年，
判决一出，群众无不交口称赞。关于马锡五审判，以下哪一说法是错误的？\n
A：群众不能理解的判决，很难具有公信力\n
B:司法裁判要坚持走群众路线\n
C:马锡五知民情知民意\n
D:只要群众满意，不必恪守法律\n
"""
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer_cot.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer_cot([text], return_tensors="pt").to(model_cot.device)

# conduct text completion
generated_ids = model_cot.generate(
    **model_inputs,
    max_new_tokens=50
)

content = tokenizer_cot.decode(generated_ids[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True).strip("\n")

print("content:", content)

content: D:只要群众满意，不必恪守法律


In [13]:
[model_inputs.input_ids.shape[1]:]

SyntaxError: invalid syntax (709890214.py, line 1)

In [16]:
tokenizer_cot.decode(generated_ids[0])

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n\n这是一个关于法律问题的单项选择题,请根据题目选择正确的选项(只需要给出正确选项即可)：\n\n问题：马锡五在审理“抢亲案"时发现，女方与男方两情相悦，定有婚约，女方父亲为了更多的彩礼将女儿许配另一人，男方父亲带领婚礼抢亲，马锡五判婚姻有效，男方父亲判徒刑半年，女方父亲判劳役半年，\n判决一出，群众无不交口称赞。关于马锡五审判，以下哪一说法是错误的？\n\nA：群众不能理解的判决，很难具有公信力\n\nB:司法裁判要坚持走群众路线\n\nC:马锡五知民情知民意\n\nD:只要群众满意，不必恪守法律\n\n<|im_end|>\n<|im_start|>assistant\nD:只要群众满意，不必恪守法律<|im_end|>'